# Testing & Inference

## Introduction

**Learning Objectives**
- Explaining the difference between training, testing and inference  
- Describing how a trained neural network makes predictions on unseen data  
- Computing and interpreting accuracy as a measure of model performance  
- Understanding the role of epochs in evaluating learning progress  
- Assessing whether a model is generalising or overfitting using test data  

By the end of this lesson, you will have a clear understanding of how a trained neural network is evaluated and used in practice. Building on the previous sections, we will continue focusing on the **multi-layer perceptron (MLP)**, to examine how model performance is assessed once our training has been completed.

## What is testing?

Using **handwritten digit recognition** as a running example, we will explore how a trained network makes predictions on previously unseen data and how these predictions are subsequently *compared* to *known labels*. We will begin with the **accuracy** metric, a simple and commonly used score for quantifying how well a model performs.

KEY TERMS BOX

**Accuracy** is a *performance metric* that measures the **proportion of correct predictions** made by a model. 

It is defined as the *number of correct predictions divided by the total number of predictions*:

$$
\text{accuracy} = \frac{\text{number of correct predictions}}{\text{total number of predictions}}
$$

The schematic below illustrates this idea for the handwritten digit classification task.

![Fig. 7: Testing unlabelled data using a trained model.](./images/Fig_testing.png)

The images used for this evaluation are drawn from the labelled data available to us, but crucially are *not* the same images used during training. 

If we were to evaluate the model only on training data, we would have no reliable way of knowing whether it has genuinely learned the task or has merely memorised the examples it was shown, without the ability to generalise on unlabelled data. This issue is closely related to the problem of **overfitting**. More information can be found in the [further reading](./further-reading.html#testing---inference-resources) section of this lesson.

In practice, model performance is usually monitored on a **validation set** *during* the process of training. This allows us to observe how a model's performance *evolves* over time: both on the training data used to update the network’s parameters and on unseen data used purely for model evaluation. Tracking these trends provides valuable insight into whether the model is learning effectively, or beginning to overfit. The **test set**, on the other hand, is *held out* for final, unbiased evaluation once the model development has completed. 

Finally, the term **inference** is typically used to describe the situation in which a trained model is applied to new, unseen data for which no ground truth labels are provided. In this setting, the model’s predictions are taken as its best estimate of the correct output.

## The AND gate function

KEY TERMS BOX

An **affine transformation** in a neural network refers to a *linear* mapping followed by a bias shift, represented as:

$$
z = Wx + b
$$

- $x$ is the input vector,
- $W$ is the weight matrix,
- $b$ is the bias vector,
- $z$ is the transformed output.

It is termed *affine* (rather than simply *linear*) because of the *added bias term* $b$. In a neuron, an affine step such as this is usually followed by a *non-linear activation* function (such as: sigmoid, ReLU, tanh).

We will begin by demonstrating a **single-neuron model** (one affine transformation followed by a sigmoid activation):

$$
\hat{y} = \sigma(Wx + b)
$$

Here, $W$ denotes a matrix of weights, $b$ is a vector of bias terms, and $x$ is a vector of input values. As discussed earlier, the weights and biases of the network are learned during training through **gradient descent**.

Our goal is to construct a single neuron that can learn the behaviour of an **AND gate**. The term 'AND gate' refers to a basic logical operation that outputs 1 only when *both* of its inputs are 1. In all other cases, the output is 0.

The truth table for an AND gate is shown below:

| first input ($x_1$) | second input ($x_2$) | output ($y$) |
|---|---|---|
| 0 | 0 | 0 |
| 0 | 1 | 0 |
| 1 | 0 | 0 |
| 1 | 1 | 1 |

In this setup, the columns $x_1$ and $x_2$ form the inputs to the network, while the column $y$ represents the desired output. We can collect these values into matrix form as follows:

$$
X =
\begin{bmatrix}
0 & 0 \\
0 & 1 \\
1 & 0 \\
1 & 1
\end{bmatrix}
$$

$$
y =
\begin{bmatrix}
0 \\
0 \\
0 \\
1
\end{bmatrix}
$$

Each row of $X$ corresponds to one possible input combination for the AND gate, and the matching entry in $y$ gives the correct output for that input.

KEY TERMS BOX

An **epoch** is one complete pass through the **entire training dataset** during the learning process. As neural nets require a lot of data, it is often processed in **batches** or **chunks**. At the end of an epoch all training samples will have been processed, the loss computed, and then used to update the parameters. Once that is completed, the process starts again until the declared number of epochs expires.

KEY TERMS BOX

The **learning rate** is a *hyperparameter* that controls **how large a step the model takes** when updating its parameters during training. It determines *how quickly the model learns*, with larger values leading to faster but potentially unstable updates, and smaller values leading to slower but more stable learning.

The code below trains a simple neural network to predict the correct output $y$, given the inputs $x1$ and $x2$. 

The code defines variables for `epochs` and `learning_rate`, so these can be easily adjusted, and the code cell re-run. A prediction and loss calculation are performed at each epoch, and the code uses a conditional statement and the modulus operator `%` to print the loss and epoch number, every 500 epochs.

In [ ]:
# Import needed modules
import numpy as np
import torch
import matplotlib.pyplot as plt

In [ ]:
# Fix the numpy and torch seeds for reproducibility
# Parameters are randomised by default, therefore we need to fix them
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

We will define some data for our model to learn. As there are only **4** possible combinations the *training* and *test* data will be identical, but in different orders.

In [ ]:
X_train = np.array(
    [[0, 0],
    [0, 1],
    [1, 0],
    [1, 1]], dtype=np.float32)

y_train = np.array([[0], [0], [0], [1]], dtype=np.float32)

X_test = np.array(
    [[1, 0],
    [0, 0],
    [1, 1],
    [0, 1]], dtype=np.float32)

y_test = np.array([[0], [0], [1], [0]], dtype=np.float32)

# Convert the numpy arrays to PyTorch tensors:
X_train = torch.from_numpy(X_train) 
y_train = torch.from_numpy(y_train)

X_test = torch.from_numpy(X_test) 
y_test = torch.from_numpy(y_test)

In [ ]:
# Define the training parameters and model input/output

# The number of epochs is the number of times the training data is passed through the single-layer neural network:
epochs = 5000

# The learning rate is the step size for the gradient descent:
learning_rate = 0.1

# Number of features - 2 in this case
input_size = len(X_train[0])

# Number of outputs - 1 in this case (on/off)
output_size = 1

The *loop* below is a *manual* implementation of some of **PyTorch's** training process, much of this will be not needed in practice, but will serve to help you follow what is happening in the background.

Breaking down the training process:

1. A prediction is made with *untrained* weights
2. Predicted probabilities of belonging to a certain class are computed using the *sigmoid* function
3. Predicted classes are then obtained by *thresholding* (or rounding) those probabilities
4. **Loss** is calculated
5. Loss is used to *update weights* in the *backward pass*

In [ ]:
losses = []
w = torch.randn(input_size, output_size, requires_grad=True)
b = torch.randn(1, 1, requires_grad=True)

for e in range(epochs):

    # Calculate the output of the neuron:
    y_pred = torch.matmul(X_train, w) + b

    # Apply a sigmoid activation function to the output of the neuron:
    y_pred = torch.sigmoid(y_pred)

    # Calculate the loss:
    loss = (y_pred - y_train).pow(2).sum() # MSE loss

    # Print the loss every 500 epochs
    if e % 500 == 0:
        print(f'Epoch: {e} | loss: {loss.item():.4f}')
    losses.append(loss.item())

    # Calculate the gradients of the loss with respect to the weights and biases:
    loss.backward()

    # Update the weights and biases:
    with torch.no_grad():
        
        w -= learning_rate * w.grad
        b -= learning_rate * b.grad

        # Zero the gradients. Otherwise, they will accumulate:
        w.grad.zero_()
        b.grad.zero_()

As we saw in the previous section, as the model is trained, the loss decreases. As visualised below:

In [ ]:
# Plot the loss over time:
plt.plot(losses)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss over time')

plt.show()
# This shows that the loss is decreasing over time, 
# which means that the neural network is learning.

PRACTICE EXERCISE

In the code block given above, we train a neural network to learn the AND gate function with a **learning rate of 0.1**.

- Copy the AND gate training code from the cell above into the cell below

- Change `learning_rate = 0.1` to `learning_rate = 0.01`

- You may wish to reduce `epochs` to `2000` to speed up your experiments

- Run the code and observe the final loss values and the shape of the loss curve.

- Try again, setting different values for the `learning_rate` variable, such as `0.5` or `1.0`

**Questions**:

1. Does the model still learn the AND gate correctly with a very small learning rate (e.g. 0.01)?

2. What happens to the loss curve when the learning rate is too high (e.g. 1.0 or 2.0)?

3. Which learning rate value gives the best balance between speed of learning and stability?

In [ ]:
# Download this notebook and write your code here.

###     

Now that we have a trained model, we can assess its *performance*. First, we obtain the predictions on the *test* dataset:

In [ ]:
# Calculate the output of the neuron:
y_pred = torch.matmul(X_test, w) + b

# Apply a sigmoid activation function to the output of the neuron:
y_pred = torch.sigmoid(y_pred)
# For efficiency once trained/predicted, tensors should be converted
# back to numpy arrays
y_pred = y_pred.detach().numpy() 
print(y_pred)

The *sigmoid* activation function places the values *near* to **1** for the positive class and **0** for the negative class. Using *numpy's* round function we can convert them to the class.

In [ ]:
y_class = np.round(y_pred)
print(y_class)

Next we can obtain the accuracy of the model on the test dataset and also visually inspect the output to see how well our trained model solves the task:

In [ ]:
from sklearn.metrics import accuracy_score

print(f"Accuracy: {accuracy_score(y_test, y_class) * 100}%")

In [ ]:
# Visualise the real labels of the AND gate to the predicted labels from the perceptron:

# Make two subplots, side by side:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

# Plot the original data:
ax1.scatter(X_test[:, 0], X_test[:,1], c=y_test.numpy(), s=200)
ax1.set_xlabel('x1')
ax1.set_ylabel('x2')

# Plot the predicted data:
ax2.scatter(X_test[:, 0], X_test[:,1], c=y_class, s=200)
ax2.set_xlabel('x1')
ax2.set_ylabel('x2')

# Add a title to each subplot:
ax1.set_title('Original data')
ax2.set_title('Predicted data')

# Add a title to the figure:
fig.suptitle('A layer of neurons learning an AND gate')

# Make a manual legend:
from matplotlib.lines import Line2D
from matplotlib import cm
import matplotlib.colors as colors

norm = colors.Normalize(vmin=0, vmax=1)
rgba_color_0 = cm.viridis(norm(0.01)) 
rgba_color_1 = cm.viridis(norm(.99)) 
line1 = Line2D([], [], color="white", marker='o', markerfacecolor=rgba_color_0, markersize=15)
line2 = Line2D([], [], color="white", marker='o', markerfacecolor=rgba_color_1, markersize=15)

plt.legend((line1, line2), ('OFF', 'ON'), numpoints=1, loc=1, bbox_to_anchor=(1.3, 1.01))
plt.tight_layout()
plt.show()

As can be seen from the plot, the predictions $y$ of the simple network match the ground-truth data for the same inputs $x1, x2$

NOTE BOX

This problem is useful for demonstrating training and testing mechanics. Note that the AND gate is **linearly separable**, so it can be solved by a single neuron with a linear decision boundary (plus a threshold). Non-linear activations, however, become essential for tasks that are not linearly separable (for example, XOR).

## Summary

We have demonstrated how to train and test a basic **MLP** model. Next, we will apply this knowledge to a more complex problem, implementing a **MLP** model using the full power of **PyTorch**.